## Tesis de Grado Machine Learning

- Alexis Carrera


## Cargar Base De Datos

## Instalar e importar bibliotecas


In [33]:
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, RichProgressBar
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pytorch_lightning.loggers import CSVLogger
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt


# Analizo la rentabilidad del S&P500

In [34]:
resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")

In [35]:
resultados_df

,Ticker,Pred_LSTM,Pred_GRU,Pred_TFT,Pred_Average,Rentabilidad,Dif_LSTM,Dif_GRU,Dif_TFT,Dif_Average,Total_Capitalization,Sector
0,NVDA,-10.282170,9.028559,-0.638273,-2.562466,12.021585,-22.303755,-2.993026,-12.659858,-14.584050,4930024243200,technology
1,AAPL,-9.314780,-4.761306,-6.071364,-7.106719,23.018155,-32.332936,-27.779461,-29.089520,-30.124874,3995082424320,technology
2,MSFT,10.414615,7.832367,7.816844,8.859057,-0.418326,10.832940,8.250693,8.235170,9.277383,3848559656960,technology
3,GOOGL,-4.382843,7.399686,-3.817669,-1.800268,43.212649,-47.595492,-35.812963,-47.030318,-45.012916,3396526145536,communication-services
4,GOOG,-4.529490,7.313377,-3.087580,-1.584153,42.980489,-47.509980,-35.667112,-46.068069,-44.564642,3396526145536,communication-services
...,...,...,...,...,...,...,...,...,...,...,...,...
497,MOH,12.059813,7.184485,12.634867,11.314769,-1.251614,13.311427,8.436100,13.886481,12.566383,7867283968,healthcare
498,MTCH,-12.681130,-4.640359,-7.547727,-9.019615,-10.542651,-2.138480,5.902292,2.994924,1.523036,7781720064,communication-services
499,HSIC,19.032873,7.843695,9.531639,12.994544,-4.097118,23.129991,11.940813,13.628757,17.091662,7664162816,healthcare
500,MHK,-11.548303,-3.567759,-9.050042,-8.952890,-5.653796,-5.894506,2.086037,-3.396246,-3.299093,7025379328,consumer-cyclical


In [36]:
import pandas as pd

resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")
# --- Filtrar: quitar sector technology ---
# resultados_df = resultados_df[resultados_df["Sector"] != "technology"]

print(len(resultados_df))
# --- Paso 1: calcular el total de capitalización ---
total_cap = resultados_df["Total_Capitalization"].sum()

# --- Paso 2: calcular el peso de cada acción ---
resultados_df["Peso"] = resultados_df["Total_Capitalization"] / total_cap

# --- Paso 3: calcular los valores ponderados ---
resultados_df["Rentabilidad_Ponderada"] = resultados_df["Peso"] * resultados_df["Rentabilidad"]
resultados_df["Prediccion_LSTM"] = resultados_df["Peso"] * resultados_df["Pred_LSTM"]
resultados_df["Prediccion_GRU"] = resultados_df["Peso"] * resultados_df["Pred_GRU"]
resultados_df["Prediccion_TFT"] = resultados_df["Peso"] * resultados_df["Pred_TFT"]
resultados_df["Prediccion_Ponderada"] = resultados_df["Peso"] * resultados_df["Pred_Average"]


# --- Paso 4: sumar para obtener el “índice S&P500 simulado” ---
sp500_rentabilidad = resultados_df["Rentabilidad_Ponderada"].sum()
sp500_prediccion_lstm = resultados_df["Prediccion_LSTM"].sum()
sp500_prediccion_gru = resultados_df["Prediccion_GRU"].sum()
sp500_prediccion_tft = resultados_df["Prediccion_TFT"].sum()
sp500_prediccion = resultados_df["Prediccion_Ponderada"].sum()

# --- Mostrar resultados ---
print("📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)")
print(f"Rentabilidad ponderada total: {sp500_rentabilidad:.4f}")
print(f"Predicción lstm ponderada total:   {sp500_prediccion_lstm:.4f}")
print(f"Predicción gru ponderada total:   {sp500_prediccion_gru:.4f}")
print(f"Predicción tft ponderada total:   {sp500_prediccion_tft:.4f}")
print(f"Predicción ponderada total:   {sp500_prediccion:.4f}")



502
📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)
Rentabilidad ponderada total: 12.6376
Predicción lstm ponderada total:   0.9995
Predicción gru ponderada total:   4.8686
Predicción tft ponderada total:   2.4026
Predicción ponderada total:   2.3346


# S&P500 SIN TECNOLOGIA

In [37]:
resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")
resultados_df = resultados_df[resultados_df["Sector"] != "technology"]
resultados_df = resultados_df[resultados_df["Ticker"] != "AMZN"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOG"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOGL"]
resultados_df = resultados_df[resultados_df["Ticker"] != "META"]
resultados_df = resultados_df[resultados_df["Ticker"] != "TSLA"]

In [38]:
resultados_df

,Ticker,Pred_LSTM,Pred_GRU,Pred_TFT,Pred_Average,Rentabilidad,Dif_LSTM,Dif_GRU,Dif_TFT,Dif_Average,Total_Capitalization,Sector
9,BRK-B,2.100343,0.074925,1.330930,1.387494,3.482351,-1.382008,-3.407426,-2.151421,-2.094857,1029917966336,financial-services
10,JPM,8.820821,16.882097,4.821827,8.833479,8.953988,-0.133167,7.928109,-4.132161,-0.120510,846930903040,financial-services
11,WMT,-2.205799,-4.443308,-3.847119,-3.309829,-1.652138,-0.553661,-2.791170,-2.194981,-1.657691,806693109760,consumer-defensive
12,LLY,16.015469,-3.436917,9.506972,9.521593,34.936686,-18.921217,-38.373603,-25.429714,-25.415093,772275896320,healthcare
14,V,11.131309,5.520163,10.176202,9.627037,2.785255,8.346053,2.734907,7.390947,6.841782,658738774016,financial-services
...,...,...,...,...,...,...,...,...,...,...,...,...
497,MOH,12.059813,7.184485,12.634867,11.314769,-1.251614,13.311427,8.436100,13.886481,12.566383,7867283968,healthcare
498,MTCH,-12.681130,-4.640359,-7.547727,-9.019615,-10.542651,-2.138480,5.902292,2.994924,1.523036,7781720064,communication-services
499,HSIC,19.032873,7.843695,9.531639,12.994544,-4.097118,23.129991,11.940813,13.628757,17.091662,7664162816,healthcare
500,MHK,-11.548303,-3.567759,-9.050042,-8.952890,-5.653796,-5.894506,2.086037,-3.396246,-3.299093,7025379328,consumer-cyclical


In [39]:
import pandas as pd

resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")

# --- Filtrar: quitar sector technology ---
resultados_df = resultados_df[resultados_df["Sector"] != "technology"]
resultados_df = resultados_df[resultados_df["Ticker"] != "AMZN"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOG"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOGL"]
resultados_df = resultados_df[resultados_df["Ticker"] != "META"]
resultados_df = resultados_df[resultados_df["Ticker"] != "TSLA"]

print(len(resultados_df))
# --- Paso 1: calcular el total de capitalización ---
total_cap = resultados_df["Total_Capitalization"].sum()

# --- Paso 2: calcular el peso de cada acción ---
resultados_df["Peso"] = resultados_df["Total_Capitalization"] / total_cap

# --- Paso 3: calcular los valores ponderados ---
resultados_df["Rentabilidad_Ponderada"] = resultados_df["Peso"] * resultados_df["Rentabilidad"]
resultados_df["Prediccion_LSTM"] = resultados_df["Peso"] * resultados_df["Pred_LSTM"]
resultados_df["Prediccion_GRU"] = resultados_df["Peso"] * resultados_df["Pred_GRU"]
resultados_df["Prediccion_TFT"] = resultados_df["Peso"] * resultados_df["Pred_TFT"]
resultados_df["Prediccion_Ponderada"] = resultados_df["Peso"] * resultados_df["Pred_Average"]


# --- Paso 4: sumar para obtener el “índice S&P500 simulado” ---
sp500_rentabilidad = resultados_df["Rentabilidad_Ponderada"].sum()
sp500_prediccion_lstm = resultados_df["Prediccion_LSTM"].sum()
sp500_prediccion_gru = resultados_df["Prediccion_GRU"].sum()
sp500_prediccion_tft = resultados_df["Prediccion_TFT"].sum()
sp500_prediccion = resultados_df["Prediccion_Ponderada"].sum()

# --- Mostrar resultados ---
print("📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)")
print(f"Rentabilidad ponderada total: {sp500_rentabilidad:.4f}")
print(f"Predicción lstm ponderada total:   {sp500_prediccion_lstm:.4f}")
print(f"Predicción gru ponderada total:   {sp500_prediccion_gru:.4f}")
print(f"Predicción tft ponderada total:   {sp500_prediccion_tft:.4f}")
print(f"Predicción ponderada total:   {sp500_prediccion:.4f}")


416
📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)
Rentabilidad ponderada total: 4.5217
Predicción lstm ponderada total:   2.5906
Predicción gru ponderada total:   3.1547
Predicción tft ponderada total:   2.5845
Predicción ponderada total:   2.7010


# RECOMENDACIONES PONDERADO > 0

In [40]:
import pandas as pd

resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")
# --- Filtrar: quitar sector technology ---
# resultados_df = resultados_df[resultados_df["Sector"] != "technology"]

# --- Filtrar: solo las acciones con Prediccion > 0 ---
resultados_df = resultados_df[resultados_df["Pred_Average"] > 0]

print(len(resultados_df))
# --- Paso 1: calcular el total de capitalización ---
total_cap = resultados_df["Total_Capitalization"].sum()

# --- Paso 2: calcular el peso de cada acción ---
resultados_df["Peso"] = resultados_df["Total_Capitalization"] / total_cap

# --- Paso 3: calcular los valores ponderados ---
resultados_df["Rentabilidad_Ponderada"] = resultados_df["Peso"] * resultados_df["Rentabilidad"]
resultados_df["Prediccion_LSTM"] = resultados_df["Peso"] * resultados_df["Pred_LSTM"]
resultados_df["Prediccion_GRU"] = resultados_df["Peso"] * resultados_df["Pred_GRU"]
resultados_df["Prediccion_TFT"] = resultados_df["Peso"] * resultados_df["Pred_TFT"]
resultados_df["Prediccion_Ponderada"] = resultados_df["Peso"] * resultados_df["Pred_Average"]


# --- Paso 4: sumar para obtener el “índice S&P500 simulado” ---
sp500_rentabilidad = resultados_df["Rentabilidad_Ponderada"].sum()
sp500_prediccion_lstm = resultados_df["Prediccion_LSTM"].sum()
sp500_prediccion_gru = resultados_df["Prediccion_GRU"].sum()
sp500_prediccion_tft = resultados_df["Prediccion_TFT"].sum()
sp500_prediccion = resultados_df["Prediccion_Ponderada"].sum()

# --- Mostrar resultados ---
print("📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)")
print(f"Rentabilidad ponderada total: {sp500_rentabilidad:.4f}")
print(f"Predicción lstm ponderada total:   {sp500_prediccion_lstm:.4f}")
print(f"Predicción gru ponderada total:   {sp500_prediccion_gru:.4f}")
print(f"Predicción tft ponderada total:   {sp500_prediccion_tft:.4f}")
print(f"Predicción ponderada total:   {sp500_prediccion:.4f}")

321
📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)
Rentabilidad ponderada total: 9.1906
Predicción lstm ponderada total:   6.9963
Predicción gru ponderada total:   7.6696
Predicción tft ponderada total:   6.9678
Predicción ponderada total:   7.1196


# RECOMENDACIONES PONDERADO > 0 SIN TECNOLOGIA

In [41]:
import pandas as pd
resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")

# --- Filtrar: quitar sector technology ---
resultados_df = resultados_df[resultados_df["Sector"] != "technology"]
resultados_df = resultados_df[resultados_df["Ticker"] != "AMZN"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOG"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOGL"]
resultados_df = resultados_df[resultados_df["Ticker"] != "META"]
resultados_df = resultados_df[resultados_df["Ticker"] != "TSLA"]

# --- Filtrar: solo las acciones con Prediccion > 0 ---
resultados_df = resultados_df[resultados_df["Pred_Average"] > 0]

print(len(resultados_df))
# --- Paso 1: calcular el total de capitalización ---
total_cap = resultados_df["Total_Capitalization"].sum()

# --- Paso 2: calcular el peso de cada acción ---
resultados_df["Peso"] = resultados_df["Total_Capitalization"] / total_cap

# --- Paso 3: calcular los valores ponderados ---
resultados_df["Rentabilidad_Ponderada"] = resultados_df["Peso"] * resultados_df["Rentabilidad"]
resultados_df["Prediccion_LSTM"] = resultados_df["Peso"] * resultados_df["Pred_LSTM"]
resultados_df["Prediccion_GRU"] = resultados_df["Peso"] * resultados_df["Pred_GRU"]
resultados_df["Prediccion_TFT"] = resultados_df["Peso"] * resultados_df["Pred_TFT"]
resultados_df["Prediccion_Ponderada"] = resultados_df["Peso"] * resultados_df["Pred_Average"]


# --- Paso 4: sumar para obtener el “índice S&P500 simulado” ---
sp500_rentabilidad = resultados_df["Rentabilidad_Ponderada"].sum()
sp500_prediccion_lstm = resultados_df["Prediccion_LSTM"].sum()
sp500_prediccion_gru = resultados_df["Prediccion_GRU"].sum()
sp500_prediccion_tft = resultados_df["Prediccion_TFT"].sum()
sp500_prediccion = resultados_df["Prediccion_Ponderada"].sum()

# --- Mostrar resultados ---
print("📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)")
print(f"Rentabilidad ponderada total: {sp500_rentabilidad:.4f}")
print(f"Predicción lstm ponderada total:   {sp500_prediccion_lstm:.4f}")
print(f"Predicción gru ponderada total:   {sp500_prediccion_gru:.4f}")
print(f"Predicción tft ponderada total:   {sp500_prediccion_tft:.4f}")
print(f"Predicción ponderada total:   {sp500_prediccion:.4f}")

253
📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)
Rentabilidad ponderada total: 5.7159
Predicción lstm ponderada total:   7.4774
Predicción gru ponderada total:   6.9755
Predicción tft ponderada total:   6.4719
Predicción ponderada total:   6.9748


# CONSUMER CYCLICAL

In [42]:
resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")
resultados_df = resultados_df[resultados_df["Sector"] == "consumer-cyclical"]
resultados_df = resultados_df[resultados_df["Ticker"] != "AMZN"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOG"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOGL"]
resultados_df = resultados_df[resultados_df["Ticker"] != "META"]
resultados_df = resultados_df[resultados_df["Ticker"] != "TSLA"]
len(resultados_df)

52

## S&P500 solo consumer-defensive

In [43]:
import pandas as pd

resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")

# --- Filtrar: quitar sector technology ---
resultados_df = resultados_df[resultados_df["Sector"] == "consumer-defensive"]
resultados_df = resultados_df[resultados_df["Ticker"] != "AMZN"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOG"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOGL"]
resultados_df = resultados_df[resultados_df["Ticker"] != "META"]
resultados_df = resultados_df[resultados_df["Ticker"] != "TSLA"]

print(len(resultados_df))
# --- Paso 1: calcular el total de capitalización ---
total_cap = resultados_df["Total_Capitalization"].sum()

# --- Paso 2: calcular el peso de cada acción ---
resultados_df["Peso"] = resultados_df["Total_Capitalization"] / total_cap

# --- Paso 3: calcular los valores ponderados ---
resultados_df["Rentabilidad_Ponderada"] = resultados_df["Peso"] * resultados_df["Rentabilidad"]
resultados_df["Prediccion_LSTM"] = resultados_df["Peso"] * resultados_df["Pred_LSTM"]
resultados_df["Prediccion_GRU"] = resultados_df["Peso"] * resultados_df["Pred_GRU"]
resultados_df["Prediccion_TFT"] = resultados_df["Peso"] * resultados_df["Pred_TFT"]
resultados_df["Prediccion_Ponderada"] = resultados_df["Peso"] * resultados_df["Pred_Average"]


# --- Paso 4: sumar para obtener el “índice S&P500 simulado” ---
sp500_rentabilidad = resultados_df["Rentabilidad_Ponderada"].sum()
sp500_prediccion_lstm = resultados_df["Prediccion_LSTM"].sum()
sp500_prediccion_gru = resultados_df["Prediccion_GRU"].sum()
sp500_prediccion_tft = resultados_df["Prediccion_TFT"].sum()
sp500_prediccion = resultados_df["Prediccion_Ponderada"].sum()

# --- Mostrar resultados ---
print("📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)")
print(f"Rentabilidad ponderada total: {sp500_rentabilidad:.4f}")
print(f"Predicción lstm ponderada total:   {sp500_prediccion_lstm:.4f}")
print(f"Predicción gru ponderada total:   {sp500_prediccion_gru:.4f}")
print(f"Predicción tft ponderada total:   {sp500_prediccion_tft:.4f}")
print(f"Predicción ponderada total:   {sp500_prediccion:.4f}")

37
📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)
Rentabilidad ponderada total: -4.4463
Predicción lstm ponderada total:   1.5911
Predicción gru ponderada total:   0.9128
Predicción tft ponderada total:   -0.3881
Predicción ponderada total:   0.6638


# RECOMENDACIONES PONDERADO > 0 SOLO Consumo DEFENSIVO


In [44]:
import pandas as pd
resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")

# --- Filtrar: quitar sector technology ---
resultados_df = resultados_df[resultados_df["Sector"] == "consumer-defensive"]
resultados_df = resultados_df[resultados_df["Ticker"] != "AMZN"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOG"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOGL"]
resultados_df = resultados_df[resultados_df["Ticker"] != "META"]
resultados_df = resultados_df[resultados_df["Ticker"] != "TSLA"]

# --- Filtrar: solo las acciones con Prediccion > 0 ---
resultados_df = resultados_df[resultados_df["Pred_Average"] > 0]

print(len(resultados_df))
# --- Paso 1: calcular el total de capitalización ---
total_cap = resultados_df["Total_Capitalization"].sum()

# --- Paso 2: calcular el peso de cada acción ---
resultados_df["Peso"] = resultados_df["Total_Capitalization"] / total_cap

# --- Paso 3: calcular los valores ponderados ---
resultados_df["Rentabilidad_Ponderada"] = resultados_df["Peso"] * resultados_df["Rentabilidad"]
resultados_df["Prediccion_LSTM"] = resultados_df["Peso"] * resultados_df["Pred_LSTM"]
resultados_df["Prediccion_GRU"] = resultados_df["Peso"] * resultados_df["Pred_GRU"]
resultados_df["Prediccion_TFT"] = resultados_df["Peso"] * resultados_df["Pred_TFT"]
resultados_df["Prediccion_Ponderada"] = resultados_df["Peso"] * resultados_df["Pred_Average"]


# --- Paso 4: sumar para obtener el “índice S&P500 simulado” ---
sp500_rentabilidad = resultados_df["Rentabilidad_Ponderada"].sum()
sp500_prediccion_lstm = resultados_df["Prediccion_LSTM"].sum()
sp500_prediccion_gru = resultados_df["Prediccion_GRU"].sum()
sp500_prediccion_tft = resultados_df["Prediccion_TFT"].sum()
sp500_prediccion = resultados_df["Prediccion_Ponderada"].sum()

# --- Mostrar resultados ---
print("📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)")
print(f"Rentabilidad ponderada total: {sp500_rentabilidad:.4f}")
print(f"Predicción lstm ponderada total:   {sp500_prediccion_lstm:.4f}")
print(f"Predicción gru ponderada total:   {sp500_prediccion_gru:.4f}")
print(f"Predicción tft ponderada total:   {sp500_prediccion_tft:.4f}")
print(f"Predicción ponderada total:   {sp500_prediccion:.4f}")

16
📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)
Rentabilidad ponderada total: -5.3982
Predicción lstm ponderada total:   10.2275
Predicción gru ponderada total:   8.4352
Predicción tft ponderada total:   4.5672
Predicción ponderada total:   7.6049


# S&P500 SIN TECNOLOGIA Y Servicios de comunicacion

In [45]:
import pandas as pd

resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")

# --- Filtrar: quitar sector technology ---
resultados_df = resultados_df[resultados_df["Sector"] != "technology"]
resultados_df = resultados_df[resultados_df["Sector"] != "communication-services"]
resultados_df = resultados_df[resultados_df["Ticker"] != "AMZN"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOG"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOGL"]
resultados_df = resultados_df[resultados_df["Ticker"] != "META"]
resultados_df = resultados_df[resultados_df["Ticker"] != "TSLA"]

# --- Paso 1: calcular el total de capitalización ---
total_cap = resultados_df["Total_Capitalization"].sum()

# --- Paso 2: calcular el peso de cada acción ---
resultados_df["Peso"] = resultados_df["Total_Capitalization"] / total_cap

# --- Paso 3: calcular los valores ponderados ---
resultados_df["Rentabilidad_Ponderada"] = resultados_df["Peso"] * resultados_df["Rentabilidad"]
resultados_df["Prediccion_LSTM"] = resultados_df["Peso"] * resultados_df["Pred_LSTM"]
resultados_df["Prediccion_GRU"] = resultados_df["Peso"] * resultados_df["Pred_GRU"]
resultados_df["Prediccion_TFT"] = resultados_df["Peso"] * resultados_df["Pred_TFT"]
resultados_df["Prediccion_Ponderada"] = resultados_df["Peso"] * resultados_df["Pred_Average"]


# --- Paso 4: sumar para obtener el “índice S&P500 simulado” ---
sp500_rentabilidad = resultados_df["Rentabilidad_Ponderada"].sum()
sp500_prediccion_lstm = resultados_df["Prediccion_LSTM"].sum()
sp500_prediccion_gru = resultados_df["Prediccion_GRU"].sum()
sp500_prediccion_tft = resultados_df["Prediccion_TFT"].sum()
sp500_prediccion = resultados_df["Prediccion_Ponderada"].sum()

# --- Mostrar resultados ---
print("📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)")
print(f"Rentabilidad ponderada total: {sp500_rentabilidad:.4f}")
print(f"Predicción lstm ponderada total:   {sp500_prediccion_lstm:.4f}")
print(f"Predicción gru ponderada total:   {sp500_prediccion_gru:.4f}")
print(f"Predicción tft ponderada total:   {sp500_prediccion_tft:.4f}")
print(f"Predicción ponderada total:   {sp500_prediccion:.4f}")

📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)
Rentabilidad ponderada total: 4.5782
Predicción lstm ponderada total:   2.5563
Predicción gru ponderada total:   3.0564
Predicción tft ponderada total:   2.5366
Predicción ponderada total:   2.6484


# RECOMENDACIONES PONDERADO > 0 SIN TECNOLOGIA Y Servicios de comunicacion

In [46]:
import pandas as pd
resultados_df = pd.read_csv("./resultados_predicciones_completas.csv")

# --- Filtrar: quitar sector technology ---
resultados_df = resultados_df[resultados_df["Sector"] != "technology"]
resultados_df = resultados_df[resultados_df["Sector"] != "communication-services"]
resultados_df = resultados_df[resultados_df["Ticker"] != "AMZN"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOG"]
resultados_df = resultados_df[resultados_df["Ticker"] != "GOOGL"]
resultados_df = resultados_df[resultados_df["Ticker"] != "META"]
resultados_df = resultados_df[resultados_df["Ticker"] != "TSLA"]

# --- Filtrar: solo las acciones con Prediccion > 0 ---
resultados_df = resultados_df[resultados_df["Pred_Average"] > 0]

print(len(resultados_df))
# --- Paso 1: calcular el total de capitalización ---
total_cap = resultados_df["Total_Capitalization"].sum()

# --- Paso 2: calcular el peso de cada acción ---
resultados_df["Peso"] = resultados_df["Total_Capitalization"] / total_cap

# --- Paso 3: calcular los valores ponderados ---
resultados_df["Rentabilidad_Ponderada"] = resultados_df["Peso"] * resultados_df["Rentabilidad"]
resultados_df["Prediccion_LSTM"] = resultados_df["Peso"] * resultados_df["Pred_LSTM"]
resultados_df["Prediccion_GRU"] = resultados_df["Peso"] * resultados_df["Pred_GRU"]
resultados_df["Prediccion_TFT"] = resultados_df["Peso"] * resultados_df["Pred_TFT"]
resultados_df["Prediccion_Ponderada"] = resultados_df["Peso"] * resultados_df["Pred_Average"]


# --- Paso 4: sumar para obtener el “índice S&P500 simulado” ---
sp500_rentabilidad = resultados_df["Rentabilidad_Ponderada"].sum()
sp500_prediccion_lstm = resultados_df["Prediccion_LSTM"].sum()
sp500_prediccion_gru = resultados_df["Prediccion_GRU"].sum()
sp500_prediccion_tft = resultados_df["Prediccion_TFT"].sum()
sp500_prediccion = resultados_df["Prediccion_Ponderada"].sum()

# --- Mostrar resultados ---
print("📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)")
print(f"Rentabilidad ponderada total: {sp500_rentabilidad:.4f}")
print(f"Predicción lstm ponderada total:   {sp500_prediccion_lstm:.4f}")
print(f"Predicción gru ponderada total:   {sp500_prediccion_gru:.4f}")
print(f"Predicción tft ponderada total:   {sp500_prediccion_tft:.4f}")
print(f"Predicción ponderada total:   {sp500_prediccion:.4f}")

240
📊 S&P500 simulado (sin sector technology y solo con Prediccion > 0)
Rentabilidad ponderada total: 6.1810
Predicción lstm ponderada total:   7.3481
Predicción gru ponderada total:   6.7861
Predicción tft ponderada total:   6.2939
Predicción ponderada total:   6.8140
